# Reading waterinfo observations

This notebook introduces how to use the `hydropandas` package to read, process and visualise data from the Waterinfo database. In this notebook the `ddlpy` package (https://github.com/Deltares/ddlpy) is used to access the waterinfo api. This package can be installed using `pip install rws-ddlpy`.

In [ ]:
import geopandas as gpd
import pandas as pd

import hydropandas as hpd
from hydropandas.io.waterinfo import get_locations_gdf

# enabling debug logging so we can see what happens in the background
hpd.util.get_color_logger("INFO");

In [ ]:
# settings
grootheid_code = None
locatie = "SCHOONHVN"
tmin = pd.Timestamp("2024-1-1")
tmax = pd.Timestamp("2024-1-3")
extent = (110000, 125000, 429550, 449900)  # Schoonhoven

In [ ]:
# get waterinfo observations within an extent
oc = hpd.read_waterinfo(
    extent=extent,
    grootheid_code=grootheid_code,
    locatie=locatie,
    tmin=tmin,
    tmax=tmax,
    keep_all_obs=False,
)
oc

In [ ]:
# show all measurement types within the extent
gdf_meas = get_locations_gdf()
gdf_meas[
    [
        "Grootheid.Code",
        "Grootheid.Omschrijving",
        "Groepering.Code",
        "Groepering.Omschrijving",
        "Parameter.Code",
        "Parameter.Omschrijving",
    ]
]

In [ ]:
# get data from a certain location and grootheid
o1 = hpd.WaterlvlObs.from_waterinfo(
    locatie="SCHOONHVN",
    grootheid_code="WATHTE",
    tmin=tmin,
    tmax=tmax,
    location_gdf=gdf_meas,  # specifying the location_gdf signficantly speeds up the process
)
o1

In [ ]:
# get data from a certain location and grootheid
o2 = hpd.WaterlvlObs.from_waterinfo(
    locatie="SCHOONHVN",
    grootheid_code="WATHTBRKD",
    groepering_code="NVT",
    tmin=tmin,
    tmax=tmax,
    location_gdf=gdf_meas,  # specifying the location_gdf signficantly speeds up the process
)

# plot data
ax = o1["value"].plot(ylabel=o1.unit, label=o1.name, legend=True)
o2["value"].plot(ylabel=o2.unit, label=o2.name, marker="o", legend=True, ax=ax);

In [ ]:
# get all measurement points within the Netherlands
gdf = hpd.io.waterinfo.get_locations_gdf()
gdf = hpd.io.waterinfo.get_locations_within_extent(
    gdf, extent=(482.06, 306602.42, 284182.97, 637049.52)
)
ax = gdf.plot(figsize=(10, 10))
# nlmod.plot.add_background_map(ax=ax)

### Water quality data

The Waterinfo database also contains water quality data

In [ ]:
# get chloride concentration from the Oosterschelde
o_cl = hpd.WaterlvlObs.from_waterinfo(
    locatie="OS4b",
    parameter_code="Cl",
    tmin="2024-8-25",
    tmax="2024-8-31",
    location_gdf=gdf,
)

# plot data
ax = o_cl["value"].plot(ylabel=o_cl.unit, label=o_cl.name, legend=True)

Or download all chloride measurements within a certain extent

In [ ]:
oc = hpd.read_waterinfo(
    extent=(80000, 90000, 429550, 449900),
    parameter_code="Cl",
    tmin="2024-8-1",
    tmax="2024-8-10",
)
oc.plots.interactive_map()

##

## Find selection criteria

Very often you don't know exactly the names of the location, grootheid_code, groepering_code or parameter_code. To get the data that you want you can follow these steps:
1. get a geodataframe with all the locations in the extent
2. query the geodataframe to find either a location/grootheid_code/groepering_code or parameter_code
3. call `read_waterinfo` with your selection criteria and a tmin and tmax value

In [ ]:
# 1 download geodataframe with all measurement points in your extent
gdf = hpd.io.waterinfo.get_locations_gdf()
gdf_locatie = hpd.io.waterinfo.get_locations_within_extent(
    gdf, extent=(80000, 90000, 429550, 449900)
)

In [ ]:
# 2 query the GeoDataFrame

# print unique names
print(f"Unique values of Grootheid Code: \n{gdf_locatie['Grootheid.Code'].unique()}\n")
print(
    f"Unique values of Groepering Code: \n{gdf_locatie['Groepering.Code'].unique()}\n"
)
print(f"Unique values of Groepering Code: \n{gdf_locatie['Parameter.Code'].unique()}\n")

# plot locations
ax = gdf_locatie.plot("Naam", figsize=(16, 6), legend=True)
try:
    import contextily as cx

    cx.add_basemap(ax=ax, crs=28992, alpha=0.5)
except ImportError:
    print("to add a basmap please install contextily using: pip install contextily")

In [ ]:
# 3 read data for selection criteria
oc = hpd.read_waterinfo(
    parameter_code="Cl", tmin="2024-8-1", tmax="2024-8-10", location_gdf=gdf_locatie
)
oc

## Retrieving data from WaterWebservices 

In the future, the WaterWebservices will become available. Currently (2024-03-20), they are not yet working. When retrieving measurements, it is always indicated that the maximum number of measurements is exceeded. Even when retrieving measurements for only one day.

Useful information:
- https://waterwebservices.beta.rijkswaterstaat.nl/test/swagger-ui/index.html#/
- https://rijkswaterstaatdata.nl/projecten/beta-waterwebservices/


In [ ]:
import json

import requests
from shapely.geometry import Point

# import nlmod

In [ ]:
# request catalogus REST API
url = "https://waterwebservices.beta.rijkswaterstaat.nl/test/METADATASERVICES/OphalenCatalogus"
body = {"CatalogusFilter": {"Compartimenten": True, "Grootheden": True}}
headers = {"content-type": "application/json"}
r = requests.post(url, data=json.dumps(body), headers=headers)

out = r.json()

In [ ]:
# plot locations from REST API
geometries = [Point(loc["Lon"], loc["Lat"]) for loc in out["LocatieLijst"]]
gdf = gpd.GeoDataFrame(out["LocatieLijst"], geometry=geometries, crs=4258)
gdf.to_crs(28992, inplace=True)
# extent_nl_poly = nlmod.util.extent_to_polygon(
#     [482.06, 306602.42, 284182.97, 637049.52]
# )
# gdf = gdf.loc[gdf.within(extent_nl_poly)]
# ax = gdf.plot(figsize=(10, 10))
# nlmod.plot.add_background_map(ax=ax)

In [ ]:
# read REST API using an extent (Schoonhoven zuid-west)
# extent_schoon = nlmod.util.polygon_from_extent((117850, 118180, 439550, 439900))
# gdf_schoon = gdf.loc[gdf.within(extent_schoon.buffer(10000))]
# ax = gdf_schoon.plot("Code", figsize=(10, 10), legend=True)
# nlmod.plot.add_background_map(ax=ax, alpha=0.5)

In [ ]:
# kies code en laat zien welke parameters erbij horen
code = "schoonhoven"

df_meta_locatie = pd.DataFrame(out["AquoMetadataLocatieLijst"]).set_index(
    "Locatie_MessageID"
)
df_meta = pd.DataFrame(out["AquoMetadataLijst"])

locatie_message_id = gdf.loc[gdf["Code"] == code, "Locatie_MessageID"].iloc[0]

AquoMetaData_MessageIDs = df_meta_locatie.loc[
    locatie_message_id, "AquoMetaData_MessageID"
]
if isinstance(AquoMetaData_MessageIDs, int):
    AquoMetaData_MessageIDs = [AquoMetaData_MessageIDs]
df_meta.loc[AquoMetaData_MessageIDs]

In [ ]:
# request om metingen op te halen
code = "ameland.nes"
aquometadata_message_id = 6

url = "https://waterwebservices.beta.rijkswaterstaat.nl/test/ONLINEWAARNEMINGENSERVICES/OphalenWaarnemingen"
body = {
    "Locatie": {"Code": code},
    "AquoPlusWaarnemingMetadata": {
        "AquoMetadata": {
            "Compartiment": {
                "Code": df_meta.loc[aquometadata_message_id, "Compartiment"]["Code"]
            },
            "Grootheid": {
                "Code": df_meta.loc[aquometadata_message_id, "Grootheid"]["Code"]
            },
        }
    },
    "Periode": {
        "Begindatumtijd": "2024-06-01T00:00:00.000+01:00",
        "Einddatumtijd": "2025-01-01T00:00:00.000+01:00",
    },
}
headers = {"content-type": "application/json"}
r = requests.post(url, data=json.dumps(body), headers=headers)
if r.status_code == 204:
    print("No data available")
else:
    meas = r.json()
    print(meas)

In [ ]:
# andere poging
code = "ameland.nes"
aquometadata_message_id = 6

url = "https://waterwebservices.beta.rijkswaterstaat.nl/test/ONLINEWAARNEMINGENSERVICES/OphalenWaarnemingen"
body = {
    "Locatie": {"Code": code},
    "AquoPlusWaarnemingMetadata": {
        "AquoMetadata": {
            "ProcesType": "verwachting",
            "Grootheid": {
                "Code": df_meta.loc[aquometadata_message_id, "Grootheid"]["Code"]
            },
        }
    },
    "Periode": {
        "Begindatumtijd": "2024-01-01T00:00:00.000+01:00",
        "Einddatumtijd": "2024-01-02T00:00:00.000+01:00",
    },
}
headers = {"content-type": "application/json"}
r = requests.post(url, data=json.dumps(body), headers=headers)
if r.status_code == 204:
    print("No data available")
else:
    meas = r.json()
    print(meas)